In [220]:
# Import librarys python
import pandas as pd
import glob
from datetime import datetime
import numpy as np
from pyspark import StorageLevel
from pyspark.sql.types import StructField, DecimalType, StringType, IntegerType, DoubleType, FloatType, DateType, TimestampType
from pyspark.sql import functions as sf, Window as w
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [221]:
spark           = SparkSession.builder.master("local[*]").appName("customerPortfolio").getOrCreate()
input_directory = '/home/pydev/workflow/dt_customer_portfolio/data/customerPortfolio.csv'

df_Data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("sep", ",").load(input_directory)
df_Data = df_Data.withColumn('Key_Fact', sf.concat(sf.col('cod_cia'), sf.lit('-'),sf.col('cod_co'), sf.lit('-'), sf.col('cod_doc'), sf.col('consec_doc')))

# w = w.orderBy(sf.desc('Key_Fact'))
# df_Data = df_Data.withColumn('eq', sf.col('Key_Fact') == sf.lag('Key_Fact').over(w))

# df_Data.withColumn("lag",lag("Key_Fact",2).over(w)).show()
# df_Data.show(10)
windowSpec = w.partitionBy("rowid_tercero").orderBy("Key_Fact")
c = df_Data.withColumn("lag",lag("Key_Fact").over(windowSpec)).show()

+---------+-------+-------------------+-------------+--------+--------------------+------------+------+-----------+-------+----------+--------------------+--------------------+--------------+----------+-----------+---------+--------+-------------+-------------+
|rowid_doc|cod_cia|          fecha_doc|rowid_tercero|     nit|        razon_social|cod_sucursal|cod_co|cod_co_mvto|cod_doc|consec_doc|     linea_preveedor|       grupo_shopper|ind_naturaleza|debitos_sa|creditos_sa|    saldo|vlr_fact|     Key_Fact|          lag|
+---------+-------+-------------------+-------------+--------+--------------------+------------+------+-----------+-------+----------+--------------------+--------------------+--------------+----------+-----------+---------+--------+-------------+-------------+
| 27189029|      2|2023-02-15 00:00:00|        42780|15017227| SAKER PADILLA BORIS|         001|     5|          5|    FV2|    105064|INDUSTRIA LICORER...|                 ILC|             1| 2829595.0|        1.0|

In [222]:
print((df_Data.count(), len(df_Data.columns)))

(97324, 19)


In [223]:
# Select the columns and remove the duplicates
dropDisDF = df_Data.select(sf.col("debitos_sa").alias("debitos_sa_2"), sf.col("creditos_sa").alias("creditos_sa_2"),sf.col("saldo").alias("saldo_2"), sf.col("Key_Fact"))
dropDisDF = dropDisDF.distinct()
# Join Data

df_Data_Clean   = df_Data.join(dropDisDF, df_Data.Key_Fact == dropDisDF.Key_Fact, how="inner").drop(df_Data.Key_Fact)
# print((dropDisDF.count(), len(dropDisDF.columns)))
# dropDisDF = dropDisDF.distinct()

In [224]:
# print((dropDisDF.count(), len(dropDisDF.columns)))
print((df_Data_Clean.count(), len(df_Data_Clean.columns)))

(97334, 22)


In [225]:
df_Data_Clean.show(50)

+---------+-------+-------------------+-------------+---------+--------------------+------------+------+-----------+-------+----------+--------------------+-------------+--------------+-----------+-----------+---------+--------+------------+-------------+---------+-------------+
|rowid_doc|cod_cia|          fecha_doc|rowid_tercero|      nit|        razon_social|cod_sucursal|cod_co|cod_co_mvto|cod_doc|consec_doc|     linea_preveedor|grupo_shopper|ind_naturaleza| debitos_sa|creditos_sa|    saldo|vlr_fact|debitos_sa_2|creditos_sa_2|  saldo_2|     Key_Fact|
+---------+-------+-------------------+-------------+---------+--------------------+------------+------+-----------+-------+----------+--------------------+-------------+--------------+-----------+-----------+---------+--------+------------+-------------+---------+-------------+
| 18720123|      2|2019-12-27 00:00:00|        45522|890900608| ALMACENES EXITO S A|         018|    10|         10|    FV2|      2643|             BACARDI|   C

In [226]:
# df_Data.printSchema()

In [227]:
df_Data.show()

+---------+-------+-------------------+-------------+---------+--------------------+------------+------+-----------+-------+----------+--------------------+-------------+--------------+-----------+-----------+---------+--------+-------------+
|rowid_doc|cod_cia|          fecha_doc|rowid_tercero|      nit|        razon_social|cod_sucursal|cod_co|cod_co_mvto|cod_doc|consec_doc|     linea_preveedor|grupo_shopper|ind_naturaleza| debitos_sa|creditos_sa|    saldo|vlr_fact|     Key_Fact|
+---------+-------+-------------------+-------------+---------+--------------------+------------+------+-----------+-------+----------+--------------------+-------------+--------------+-----------+-----------+---------+--------+-------------+
| 18720123|      2|2019-12-27 00:00:00|        45522|890900608| ALMACENES EXITO S A|         018|    10|         10|    FV2|      2643|             BACARDI|   COCTELERIA|             1|  8769993.0|   156713.0|8613280.0|  111506| 2-10-FV22643|
| 18955407|      2|2020-02-1